In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
path = '/content/drive/MyDrive/datasets/insurance_data (1).csv'

import pandas as pd

df = pd.read_csv(path)
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=42)

In [4]:
print(len(X_train))
X_train[:5]

22


,age,affordibility
17,58,1
22,40,1
11,28,1
13,29,0
15,55,1


In [5]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [6]:
X_train_scaled[:5]

,age,affordibility
17,0.58,1
22,0.40,1
11,0.28,1
13,0.29,0
15,0.55,1


In [7]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 7ms/step - loss: 0.5863 - accuracy: 0.7727
Epoch 1833/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5863 - accuracy: 0.7727
Epoch 1834/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5862 - accuracy: 0.7727
Epoch 1835/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5862 - accuracy: 0.7727
Epoch 1836/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5861 - accuracy: 0.7727
Epoch 1837/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5861 - accuracy: 0.7727
Epoch 1838/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.5861 - accuracy: 0.7727
Epoch 1839/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5860 - accuracy: 0.7727
Epoch 1840/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5860 - accuracy: 0.7727
Epoch 1841/5000
1/1 [==============================] - 0

In [8]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 191ms/step - loss: 0.2647 - accuracy: 1.0000


[0.26470306515693665, 1.0]

In [9]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 119ms/step


array([[0.81626713],
       [0.75767535],
       [0.82367426],
       [0.18819512],
       [0.38559142],
       [0.19598222]], dtype=float32)

In [10]:
X_test_scaled

,age,affordibility
9,0.61,1
25,0.54,1
8,0.62,1
21,0.26,0
0,0.22,1
12,0.27,0


In [11]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [12]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.018309 ],
        [1.1966311]], dtype=float32),
 array([-2.766541], dtype=float32))

In [13]:
def sigmoid(x):
  import math
  return 1 / (1 + math.exp(-x))

sigmoid(18)

0.9999999847700205

In [14]:
def prediction_function(age, affordability):
  weighted_sum = coef[0]*age + coef[1]*affordability + intercept
  return sigmoid(weighted_sum)

In [15]:
prediction_function(.61, 1)

0.816267109586199

In [16]:
prediction_function(.18, 1)

0.3392553710415985

In [17]:
import numpy as np

def log_loss(y_true, y_pred):
  epsilon = 1e-15
  y_pred_new = [max(i, epsilon) for i in y_pred]
  y_pred_new = [min(i, 1-epsilon) for i in y_pred_new]
  y_pred_new = np.array(y_pred_new)
  return -np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))

In [18]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12, 0, 1]))

array([0.99999386, 0.5       , 0.73105858])

In [19]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
  # w1, w2, bias
  w1 = w2 = 1
  bias = 0
  rate = 0.5
  n = len(age)

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * affordability + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = log_loss(y_true, y_predicted)

    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
    w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted - y_true))

    bias_d = np.mean(y_predicted - y_true)

    w1 = w1 - rate * w1d
    w2 = w2 - rate * w2d
    bias = bias - rate * bias_d

    print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

    if loss <= loss_thresold:
      break

  return w1, w2, bias


In [20]:
# def gradient_descent(age, affordability, y_true, epochs):
#     # w1, w2, bias
#     w1 = w2 = 1
#     bias = 0
#     rate = 0.5
#     n = len(age)

#     for i in range(epochs):
#         weighted_sum = w1 * age + w2 * affordability + bias
#         y_predicted = sigmoid_numpy(weighted_sum)

#         loss = log_loss(y_true, y_predicted)

#         w1d = (1/n) * (age * (y_predicted - y_true)).sum()
#         w2d = (1/n) * (affordability * (y_predicted - y_true)).sum()


#### you can remove the np.transpose, and the commented method is another way of doing it

In [21]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.4631)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:0.7428288579142563
Epoch:1, w1:0.9536535852311094, w2:0.8740290167758512, bias:-0.21881533456146035, loss:0.7072146449948488
Epoch:2, w1:0.9393731039296969, w2:0.8271852202997496, bias:-0.3053620401943441, loss:0.6814881914786812
Epoch:3, w1:0.9301932588998061, w2:0.7897792032048467, bias:-0.37884372361582785, loss:0.6633428084673968
Epoch:4, w1:0.9254091137248938, w2:0.7605726653866934, bias:-0.44108236820018304, loss:0.650742850709519
Epoch:5, w1:0.9243325693598607, w2:0.738313053647322, bias:-0.49384257986251556, loss:0.6420508089402462
Epoch:6, w1:0.926333296357235, w2:0.7218280753843739, bias:-0.5387319906498417, loss:0.6360356979531208
Epoch:7, w1:0.930858097563688, w2:0.7100747303660235, bias:-0.5771558825717441, loss:0.631816485354411
Epoch:8, w1:0.9374354910317362, w2:0.7021560855322683, bias:-0.6103083840841516, loss:0.6287844495353145
Epoch:9, w1:0.9456716791005845, w2:0.6973185496313956, b

(6.729458709063027, 1.3482027165467207, -3.5831821575352403)

In [22]:
coef, intercept

(array([[5.018309 ],
        [1.1966311]], dtype=float32),
 array([-2.766541], dtype=float32))

In [23]:
X_train_scaled['affordibility']

17    1
22    1
11    1
13    0
15    1
1     0
4     1
5     1
2     1
16    0
23    1
3     0
26    1
24    1
18    0
27    1
20    1
7     0
10    1
14    1
19    1
6     0
Name: affordibility, dtype: int64

In [24]:
X_train_scaled['age']

17    0.58
22    0.40
11    0.28
13    0.29
15    0.55
1     0.25
4     0.46
5     0.56
2     0.47
16    0.25
23    0.45
3     0.52
26    0.23
24    0.50
18    0.19
27    0.46
20    0.21
7     0.60
10    0.18
14    0.49
19    0.18
6     0.55
Name: age, dtype: float64

In [25]:
np.transpose(X_train_scaled['age'])

17    0.58
22    0.40
11    0.28
13    0.29
15    0.55
1     0.25
4     0.46
5     0.56
2     0.47
16    0.25
23    0.45
3     0.52
26    0.23
24    0.50
18    0.19
27    0.46
20    0.21
7     0.60
10    0.18
14    0.49
19    0.18
6     0.55
Name: age, dtype: float64

In [26]:
# example

y_predicted = np.array([0.56, 0.78, 0.86])
y_true = np.array([0, 1, 1])

age = [52, 20, 43]

np.dot(np.transpose(age),(y_predicted - y_true))

18.700000000000006

In [47]:
class myNN:
  def __init__(self):
    self.w1 = 1
    self.w2 = 1
    self.bias = 0


  def fit(self, X, y, epochs, loss_thresold):
    self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_thresold)

  def predict(self, X_test):
    weighted_sum = self.w1* X_test['age'] + self.w2 * X_test['affordibility'] + self.bias
    return sigmoid_numpy(weighted_sum)

  def gradient_descent(self, age, affordability, y_true, epochs, loss_thresold):
    # w1, w2, bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)

    for i in range(epochs):
      weighted_sum = w1 * age + w2 * affordability + bias
      y_predicted = sigmoid_numpy(weighted_sum)

      loss = log_loss(y_true, y_predicted)

      w1d = (1/n)*np.dot(np.transpose(age),(y_predicted - y_true))
      w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted - y_true))

      bias_d = np.mean(y_predicted - y_true)

      w1 = w1 - rate * w1d
      w2 = w2 - rate * w2d
      bias = bias - rate * bias_d

      if i%50==0:
        print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")

      if loss <= loss_thresold:
        print(f"Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}")
        break

    return w1, w2, bias




In [48]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=1000, loss_thresold=0.4631)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659, bias:-0.11748951666770448, loss:0.7428288579142563
Epoch:50, w1:1.524279872239132, w2:0.8822187836689879, bias:-1.1310088596841466, loss:0.5943161377198863
Epoch:100, w1:2.2281870363373972, w2:1.021207570480777, bias:-1.5219348336892933, loss:0.5674253359204207
Epoch:150, w1:2.8788802020633866, w2:1.0918966282424585, bias:-1.8376731064036336, loss:0.5462065237139112
Epoch:200, w1:3.473286412049912, w2:1.1354958313845407, bias:-2.109494376761035, loss:0.5289749850551891
Epoch:250, w1:4.014235251285109, w2:1.168307211546194, bias:-2.3515205001257873, loss:0.5148317335261617
Epoch:300, w1:4.506491403088423, w2:1.1967920767703706, bias:-2.570834600613761, loss:0.5031414005690805
Epoch:350, w1:4.955154772161112, w2:1.2234470550440064, bias:-2.7714756232427904, loss:0.4934177492028928
Epoch:400, w1:5.365041991551451, w2:1.2491715720776855, bias:-2.956078966702517, loss:0.48528128420193234
Epoch:450, w1:5.740481994774169, w2:1.274234883839

In [49]:
customModel.predict(X_test_scaled)

9     0.866457
25    0.802014
8     0.874054
21    0.137819
0     0.319842
12    0.146012
dtype: float64

In [50]:
coef, intercept

(array([[5.018309 ],
        [1.1966311]], dtype=float32),
 array([-2.766541], dtype=float32))

In [53]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 14ms/step


array([[0.81626713],
       [0.75767535],
       [0.82367426],
       [0.18819512],
       [0.38559142],
       [0.19598222]], dtype=float32)